In [24]:
import os
import shutil
import numpy as np
import glob   
import keras.backend as K
from keras import layers
from keras.layers import Input, Add, Dense, Activation, ZeroPadding2D, BatchNormalization, Flatten, Conv2D, AveragePooling2D, MaxPooling2D, GlobalAveragePooling2D
from keras.models import Model, load_model
from keras import optimizers
from keras.preprocessing import image
from keras.utils import layer_utils
from keras.utils.data_utils import get_file
from keras.applications.imagenet_utils import preprocess_input
from IPython.display import SVG
from keras.utils.vis_utils import model_to_dot
from keras.utils import plot_model
from keras.initializers import glorot_uniform
from keras.layers import Dense, Dropout, Flatten
from keras.preprocessing.image import ImageDataGenerator,load_img, img_to_array
import scipy.misc
from keras.applications import ResNet50
from keras.applications.resnet50 import preprocess_input

from matplotlib.pyplot import imshow
%matplotlib inline
K.set_image_data_format('channels_last')
K.set_learning_phase(1)


In [12]:
# Where all dataset is there
data_dir = '/home/ubuntu/flowers/'

# Training data dir
training_dir = '/home/ubuntu/data/flowers/Train'

# Test data dir
testing_dir = '/home/ubuntu/data/flowers/Test'

# Ratio of training and testing data
train_test_ratio = 0.8 


def split_dataset_into_test_and_train_sets(all_data_dir = data_dir, training_data_dir = training_dir, testing_data_dir=testing_dir, train_test_ratio = 0.8):
    # Recreate testing and training directories
    
    if not os.path.exists(training_data_dir):
            os.mkdir(training_data_dir)

    if not os.path.exists(testing_data_dir):
            os.mkdir(testing_data_dir)               
    
    num_training_files = 0
    num_testing_files = 0


    for subdir, dirs, files in os.walk(all_data_dir):
        
        category_name = os.path.basename(subdir)

        # print(category_name + " vs " + os.path.basename(all_data_dir))
        if category_name == os.path.basename(all_data_dir):
            continue

        training_data_category_dir = training_data_dir + '/' + category_name
        testing_data_category_dir = testing_data_dir + '/' + category_name
        
        # creating subdir for each sub category
        if not os.path.exists(training_data_category_dir):
            os.mkdir(training_data_category_dir)   

        if not os.path.exists(testing_data_category_dir):
            os.mkdir(testing_data_category_dir)
            
        file_list = glob.glob(os.path.join(all_data_dir, subdir,'*.jpg'))

        print(str(category_name) + ' has ' + str(len(files)) + ' images') 
        random_set = np.random.permutation((file_list))
        
        # copy percentage of data from each category to train and test directory
        train_list = random_set[:round(len(random_set)*(train_test_ratio))] 
        test_list = random_set[-round(len(random_set)*(1-train_test_ratio)):]
        
        for lists in train_list : 
            shutil.copy(lists, training_data_dir + '/' + category_name + '/' )
            num_training_files += 1
  
        for lists in test_list : 
            shutil.copy(lists, testing_data_dir + '/' + category_name + '/' )
            num_testing_files += 1
  

    print("Processed " + str(num_training_files) + " training files.")
    print("Processed " + str(num_testing_files) + " testing files.")

In [13]:
split_dataset_into_test_and_train_sets()

sunflower has 734 images
daisy has 769 images
tulip has 984 images
dandelion has 1052 images
rose has 784 images
Processed 3458 training files.
Processed 865 testing files.


In [15]:
# Number of classes in dataset
num_classes = 5

def get_model():
    # Get base model 
    # Here we are using ResNet50 as base model
    base_model = ResNet50(weights='imagenet', include_top=False)
    
    # As we are using ResNet model only for feature extraction and not adjusting the weights
    # we freeze the layers in base model
    for layer in base_model.layers:
        layer.trainable = False
        
    # Get base model output 
    base_model_ouput = base_model.output
    
    # Adding our own layer 
    x = GlobalAveragePooling2D()(base_model_ouput)
    # Adding fully connected layer
    x = Dense(512, activation='relu')(x)
    x = Dense(num_classes, activation='softmax', name='fcnew')(x)
    
    model = Model(inputs=base_model.input, outputs=x)
    return model

In [18]:
# Get the model
model = get_model()
# Compile it
model.compile(loss='categorical_crossentropy', optimizer='sgd', metrics=['accuracy'])
# Summary of model
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, None, None, 3 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, None, None, 3 0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1 (Conv2D)                  (None, None, None, 6 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
bn_conv1 (BatchNormalization)   (None, None, None, 6 256         conv1[0][0]                      
__________________________________________________________________________________________________
activation

In [21]:
# Defining the imagedatagenerator for train and test image for pre-processing
# We don't give horizonal_flip or other preprocessing for validation data generator

image_size = 224
batch_size = 64

train_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input,
    shear_range=0.2, zoom_range=0.2, horizontal_flip=True)
valid_data_gen = ImageDataGenerator(preprocessing_function = preprocess_input)
train_generator = train_data_gen.flow_from_directory(training_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')
valid_generator = valid_data_gen.flow_from_directory(testing_dir, (image_size,image_size), batch_size=batch_size, class_mode='categorical')

Found 3458 images belonging to 5 classes.
Found 865 images belonging to 5 classes.


In [22]:
# Training the fully conncected layer for initial epochs
epochs = 5

# Training the model

model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=epochs,
    verbose=1)

Epoch 1/5
54/54 [==============================] - 59s 1s/step - loss: 0.9308 - acc: 0.6630 - val_loss: 0.5537 - val_acc: 0.8281
Epoch 2/5
54/54 [==============================] - 57s 1s/step - loss: 0.4972 - acc: 0.8337 - val_loss: 0.4225 - val_acc: 0.8594
Epoch 3/5
54/54 [==============================] - 57s 1s/step - loss: 0.3768 - acc: 0.8810 - val_loss: 0.3741 - val_acc: 0.8738
Epoch 4/5
54/54 [==============================] - 56s 1s/step - loss: 0.3397 - acc: 0.8932 - val_loss: 0.4326 - val_acc: 0.8558
Epoch 5/5
54/54 [==============================] - 55s 1s/step - loss: 0.2898 - acc: 0.9062 - val_loss: 0.3240 - val_acc: 0.8942


In [26]:
# More fine tuning the model
# Training the model after 150 layers
# Generally ResNet is good at extracting lower level features so we are not fine tuning initial layers
epochs = 10

split_at = 140
for layer in model.layers[:split_at]: layer.trainable = False
for layer in model.layers[split_at:]: layer.trainable = True
    
model.compile(optimizer='sgd', loss='categorical_crossentropy', metrics=['accuracy'])

# Choosing lower learning rate for fine-tuning
# learning rate is generally 10-1000 times lower than normal learning rate, if we are fine tuning the initial layers
sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)

model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

model.fit_generator(
    train_generator,
    steps_per_epoch=train_generator.n//batch_size,
    validation_data=valid_generator,
    validation_steps=valid_generator.n//batch_size,
    epochs=epochs,
    verbose=1)

Epoch 1/10
54/54 [==============================] - 60s 1s/step - loss: 0.2540 - acc: 0.9230 - val_loss: 0.3037 - val_acc: 0.8942
Epoch 2/10
54/54 [==============================] - 58s 1s/step - loss: 0.2345 - acc: 0.9217 - val_loss: 0.2753 - val_acc: 0.9014
Epoch 3/10
54/54 [==============================] - 58s 1s/step - loss: 0.1598 - acc: 0.9537 - val_loss: 0.2552 - val_acc: 0.9135
Epoch 4/10
54/54 [==============================] - 57s 1s/step - loss: 0.1410 - acc: 0.9543 - val_loss: 0.2575 - val_acc: 0.9111
Epoch 5/10
54/54 [==============================] - 58s 1s/step - loss: 0.1146 - acc: 0.9687 - val_loss: 0.2363 - val_acc: 0.9195
Epoch 6/10
54/54 [==============================] - 57s 1s/step - loss: 0.1036 - acc: 0.9654 - val_loss: 0.2373 - val_acc: 0.9219
Epoch 7/10
54/54 [==============================] - 56s 1s/step - loss: 0.0805 - acc: 0.9797 - val_loss: 0.2345 - val_acc: 0.9171
Epoch 8/10
54/54 [==============================] - 56s 1s/step - loss: 0.0974 - acc: 0.96

In [27]:
print('Training complete')

Training complete
